## EMA Project Diary

## Data cleaning and shaping

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt 

furl = 'EMA_data/key-stage4/Tables/SFR01_2016_LA_Tables.xlsx'
furl2 = 'EMA_data/Deprivation_Index/File_10_ID2015_Local_Authority_District_Summaries.xlsx'

df_gcse = pd.read_excel(furl, "Table LA1")
df_deprivation = pd.read_excel(furl2, "IMD")

In [ ]:
df_gcse.head(15)

In [ ]:
df_gcse.tail(15)


Spreadsheet table is not formatted for easy import into a pandas dataframe and will require a lot of cleaning, including:

- removal of empty rows
- removal of metadata
- naming of columns, as names are spread over rows 3 and 4

There are rows and columns that contain no usable data and need to be removed.

In [ ]:
#remove unwanted row/column blocks from edges

df_gcse = df_gcse.iloc[3:182, 0:12]

df_gcse.head()

In [ ]:
df_gcse.tail()

That has handled blocks of initial and final rows with no data, leaving the block with the data we want intact.

Rows 3 and 4 contain column headings, and the columns themselves are unnamed or wrongly named.  Rename, then rows 3 and 4 can be removed.

In [ ]:
df_gcse.rename(columns={
    'Table LA1: GCSE and equivalent entries and achievements of pupils at the end of key stage 4 by gender for each local authority1 and region': 'Region', 
    'Unnamed: 1': 'Region #',
    'Unnamed: 2': '# Pupils',
    'Unnamed: 3': '% 5+ A*-C',
    'Unnamed: 4': '% 5+ A*-C inc Eng & Maths GCSE',
    'Unnamed: 5': '% 5+ A*-G',     
    'Unnamed: 6': '% 5+ A*-G inc Eng & Maths GCSE',
    'Unnamed: 7': '% Any passes',  
    'Unnamed: 8': 'Av. capped GCSE & Equiv. Points Score Per Pupil',
    'Unnamed: 9': 'Av. GCSE & Equiv. Points Score Per Pupil',
    'Unnamed: 10': 'GCSE or Equv. Maths & English % entered',
    'Unnamed: 11': 'GCSE or Equv. Maths & English A*-C % achieved'
}, 
          inplace=True)

df_gcse.head(5)

In [ ]:
#remove nan rows
df_gcse = df_gcse[df_gcse['Region'].notnull()]
df_gcse.head(15)

Compare Local Authorities on these factors:
- IMD Average score
- IMD - Extent
- Income Deprivation - Average score
- Employment Deprivation - Average Score
- Education Deprivation - Average Score
- Health Deprivation - Average Score
- Crime - Average Score
- Education Deprivation - Average Score
- Barriers to housing - Average Score 
- Living Environment - Average Score
- % 5+ A*-C inc Eng & Maths GCSE
- % 5+ A*-G inc Eng & Maths GCSE
- English/Maths entered
- Eng/Math achieved
- 5+ A*-C WITHOUT both Eng and Maths
- 5+ A-G WITHOUT both Eng and Maths

The latter two will give an indication of where there is high achievement in other areas, but Maths&Eng is falling behind. Average scores and overall extent (proportion of an area's population living in nationally most deprived areas) have been chose as deprivation data points.  These provide direct rather than ranked measures of deprivation, though care should be taken with understanding their method of calculation and specific meanings and flaws, as described in materials and reports.

Identify clusters, if any, with k-means


Restart data cleaning with required data, with reference to above.

In [ ]:
df_gcse = pd.read_excel(furl, "Table LA1")
df_gcse = df_gcse.iloc[3:182, 0:12]
df_gcse.rename(columns={
    'Table LA1: GCSE and equivalent entries and achievements of pupils at the end of key stage 4 by gender for each local authority1 and region': 'Region', 
    'Unnamed: 1': 'Region #',
    'Unnamed: 3': '% 5+ A*-C',
    'Unnamed: 4': '% 5+ A*-C inc Eng & Maths GCSE',
    'Unnamed: 5': '% 5+ A*-G',     
    'Unnamed: 6': '% 5+ A*-G inc Eng & Maths GCSE', 
    'Unnamed: 10': 'GCSE or Equv. Maths & English % entered',
    'Unnamed: 11': 'GCSE or Equv. Maths & English A*-C % achieved'
}, inplace=True)

df_gcse = df_gcse[["Region", 
         "Region #", 
         "% 5+ A*-C", 
         "% 5+ A*-C inc Eng & Maths GCSE", 
         "% 5+ A*-G", 
         "% 5+ A*-G inc Eng & Maths GCSE",
         "GCSE or Equv. Maths & English % entered",
        "GCSE or Equv. Maths & English A*-C % achieved"]]

df_gcse = df_gcse[df_gcse['Region'].notnull()]

df_gcse.head(10)

Look at deprivation data.

In [ ]:
df_deprivation.head(10)

Reduce to needed columns.

In [ ]:
df_deprivation = df_deprivation[["Local Authority District code (2013)", "Local Authority District name (2013)", "IMD - Average score", "IMD - Extent"]]
df_deprivation

Are we looking at the same regions?

Check local authority district codes against each other in both tables.

In [ ]:
deprivation_extra = set(df_deprivation["Local Authority District code (2013)"].unique()) - set(df_gcse["Region #"].unique())
deprivation_extra

In [ ]:
gcse_extra = set(df_gcse["Region #"].unique()) - set(df_deprivation["Local Authority District code (2013)"].unique())
gcse_extra

There are a great deal codes present in one set not present in the other, in both directions. That doesn't quite seem right. 

In [ ]:
df_gcse_extra_regions = df_gcse[["Region", "Region #"]].loc[df_gcse['Region #'].isin(gcse_extra)]
df_gcse_extra_regions

Many of these present in the gcse data clearly represent higher level regions, not the lower level local authorities in the deprivation data. Plus two rows with no region number that need removal. A possible exception is Northumberland.  Some research points to E1* numbers belonging to regions or counties, not the lower level local authorities.  E9, E12 and E13 refer to totals in table LA1.  These can be removed.  

In [ ]:
df_gcse.drop(df_gcse[df_gcse["Region #"]=='Region/\nLocal Authority number'].index, inplace=True)
df_gcse = df_gcse[df_gcse['Region #'].notnull()]

df_gcse.drop(df_gcse[df_gcse["Region #"].str.contains('E9.*')].index, inplace=True)
df_gcse.drop(df_gcse[df_gcse["Region #"].str.contains('E12.*')].index, inplace=True)
df_gcse.drop(df_gcse[df_gcse["Region #"].str.contains('E13.*')].index, inplace=True)
df_gcse


In [ ]:
gcse_extra = set(df_gcse["Region #"].unique()) - set(df_deprivation["Local Authority District code (2013)"].unique())
gcse_extra

This leaves Northumberland and Gateshead as suspect values, and there are still a huge amount of extra values in the deprivation data.

In [ ]:
df_deprivation_extra_regions = df_deprivation[["Local Authority District name (2013)", "Local Authority District code (2013)"]].loc[df_deprivation['Local Authority District code (2013)'].isin(deprivation_extra)]
df_deprivation_extra_regions

By lucky coincidence we can see that both Northumberland and Gateshead have a different district code in this dataset.  Why?  

The remaining regions in table LA1 refer to county level regions.  These can be found in a different deprivation spreadsheet file, File_11_ID_2015_Upper-tier_Local_Authority_Summaries.xlsx. In addition, E10000002 from table LA1 refers to Buckinghamshire. In the deprivation data, the local authorities that make up Buckinghamshire can be found.  We might be looking at the wrong level of local authority, or have to merge two different levels for a complete dataset.

In [ ]:
furl3 = 'EMA_data/Deprivation_Index/File_11_ID_2015_Upper-tier_Local_Authority_Summaries.xlsx'
df_deprivation2 = pd.read_excel(furl3, "IMD")[['Upper Tier Local Authority District code (2013)', 'Upper Tier Local Authority District name (2013)', 'IMD - Average score', 'IMD - Extent']]
df_deprivation2

The upper level authorities table contains the same number of entries as our gcse table.  Coincidence?

In [ ]:
set(df_gcse["Region #"].unique()) - set(df_deprivation2["Upper Tier Local Authority District code (2013)"].unique())

In [ ]:
set(df_deprivation2["Upper Tier Local Authority District code (2013)"].unique()) - set(df_gcse["Region #"].unique())

There are far fewer mismatches here, which is hopeful.  But Northumberland and Gateshead continue to cause problems.  In LA1, they are given as E06000048 and E08000020 respectively.  In the deprivation data, they are E06000057 and E08000037.

It turns out that E06000057 is a region that contains now inactive predecessors E06000048 and most of E08000020, with E08000037 containing the remainder of E08000020.  The deprivation data is slightly more up to date on these codes, but the redistribution means that it is difficult to match them up.

E06000048 can be replaced with E06000057, and E08000020 with E08000037, with the caveat that the data may not have been collected over the exact same region in these cases.

In [ ]:
df_gcse.replace('E06000048','E06000057', inplace=True)
df_gcse.replace('E08000020','E08000037', inplace=True)

In [ ]:
set(df_gcse["Region #"].unique()) - set(df_deprivation2["Upper Tier Local Authority District code (2013)"].unique())

In [ ]:
set(df_deprivation2["Upper Tier Local Authority District code (2013)"].unique()) - set(df_gcse["Region #"].unique())

Now the two sets of authority codes match and we can move on to getting the rest of the data.

In [ ]:
furl3 = 'EMA_data/Deprivation_Index/File_11_ID_2015_Upper-tier_Local_Authority_Summaries.xlsx'
df_deprivation2 = pd.read_excel(furl3, "IMD")[['Upper Tier Local Authority District code (2013)', 'Upper Tier Local Authority District name (2013)', 'IMD - Average score', 'IMD - Extent']]
temp = pd.read_excel(furl3, "Income")[['Upper Tier Local Authority District code (2013)', 'Income - Average score']]
df_deprivation2 = df_deprivation2.merge(temp)
temp = pd.read_excel(furl3, "Employment")[['Upper Tier Local Authority District code (2013)', 'Employment - Average score']]
df_deprivation2 = df_deprivation2.merge(temp)
temp = pd.read_excel(furl3, "Education")[['Upper Tier Local Authority District code (2013)', 'Education, Skills and Training - Average score']]
df_deprivation2 = df_deprivation2.merge(temp)
temp = pd.read_excel(furl3, "Health")[['Upper Tier Local Authority District code (2013)', 'Health Deprivation and Disability - Average score']]
df_deprivation2 = df_deprivation2.merge(temp)
temp = pd.read_excel(furl3, "Crime")[['Upper Tier Local Authority District code (2013)', 'Crime - Average score']]
df_deprivation2 = df_deprivation2.merge(temp)
temp = pd.read_excel(furl3, "Barriers")[['Upper Tier Local Authority District code (2013)', 'Barriers to Housing and Services - Average score']]
df_deprivation2 = df_deprivation2.merge(temp)
temp = pd.read_excel(furl3, "Living")[['Upper Tier Local Authority District code (2013)', 'Living Environment - Average score']]
df_deprivation2 = df_deprivation2.merge(temp)
df_deprivation2.head()

Time to try to merge the two datasets.  First rename the deprivation dataset.

In [ ]:
df_deprivation2.rename(columns={'Upper Tier Local Authority District code (2013)': 'Region #',
                              'Upper Tier Local Authority District name (2013)': 'Region2'}, inplace=True)
                    
df_deprivation2.head()


In [ ]:
df_gcse.head()

In [ ]:
df_compare = df_gcse.merge(df_deprivation2, left_on='Region #', right_on='Region #')
df_compare

This dataframe contains the expected columns and 152 rows. One last check: do the region names line up?

In [ ]:
import numpy as np
df_unmatched_regions = np.where((df_compare['Region'] == df_compare['Region2']), 
                                "match", df_compare['Region #'])
set(df_unmatched_regions)

There are no mismatches in the names, so we can assume that data is referring to the same regions and there have been no other changes in region code assignments.  No need for the second set of names now.

In [ ]:
df_compare.drop(columns=['Region2'], inplace=True)
df_compare.head()

A quick look at the data overall.

In [ ]:
df_compare.describe()

Ooooops, it's not picking up all the numerical columns.

In [ ]:
df_compare.dtypes

...there is data that needs casting to the correct type.  And casting to float is not working.

In [ ]:
#df_compare.astype({'Region': 'string',
#                  'Region #': 'string',
#                  '% 5+ A*-C': 'float64',
#                  '% 5+ A*-C inc Eng & Maths GCSE': 'float64',
#                  '% 5+ A*-G': 'float64',
#                  '% 5+ A*-G inc Eng & Maths GCSE': 'float64',
#                  'GCSE or Equv. Maths & English % entered': 'float64',
#                  'GCSE or Equv. Maths & English A*-C % achieved': 'float64'}).dtypes

# Throws: ValueError: could not convert string to float: '.'

In [ ]:
set(df_compare['% 5+ A*-C'])

Lets find that dot.

In [ ]:
df_compare[df_compare['% 5+ A*-C'] == "."]

The dot means data is unavailable (https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/494073/SFR01_2016.pdf pg 27) .Better remove.

In [ ]:
df_compare.drop(df_compare[df_compare["Region #"] == "E09000001"].index, inplace=True)
df_compare[df_compare['% 5+ A*-C'] == "."]

In [ ]:
df_compare = df_compare.astype({'Region': 'string',
                  'Region #': 'string',
                  '% 5+ A*-C': 'float64',
                  '% 5+ A*-C inc Eng & Maths GCSE': 'float64',
                  '% 5+ A*-G': 'float64',
                  '% 5+ A*-G inc Eng & Maths GCSE': 'float64',
                  'GCSE or Equv. Maths & English % entered': 'float64',
                  'GCSE or Equv. Maths & English A*-C % achieved': 'float64'})

That did it.

In [ ]:
df_compare.dtypes

In [ ]:
df_compare.describe()

We also wanted to look at where GCSEs were being achieved without both Maths and English.

In [ ]:
df_compare['% 5+ A*-G excl Eng & Maths GCSE'] = df_compare['% 5+ A*-G'] - df_compare['% 5+ A*-G inc Eng & Maths GCSE']
df_compare['% 5+ A*-C excl Eng & Maths GCSE'] = df_compare['% 5+ A*-C'] - df_compare['% 5+ A*-C inc Eng & Maths GCSE']
df_compare

In [ ]:
df_compare.describe()

Now the data is ready to be analysed.

## Analysis

Run this to start from this stage skipping above:

In [ ]:
df_compare = pd.read_csv('Merged.csv')
df_gcse = pd.read_csv('KS4.csv')
df_deprivation2 = pd.read_csv('IMD.csv')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import cluster
import scipy.stats

We would expect the deprivation measures to be connected, but how?  Previous work on this has been done in TMA02 showing some correlations.  Could they be assigned into clusters?

Let's create a tool to help us use k means to cluster data, so we're not repeating a lot of code

In [ ]:
def clusterise(data, k):
    newDF = data.copy().select_dtypes(exclude=['string', 'object'])
    kmeans = cluster.KMeans(n_clusters=k)
    newDF['Cluster'] = kmeans.fit(newDF).labels_
    return newDF

And another tool to plot clustered data

In [ ]:
def plotClusters(data, ax1name, ax2name, k):
    cl = data['Cluster']
    colour = ['red', 'blue', 'green', 'cyan', 'magenta', 'yellow', 'black']
    for x in range(k):
        plt.scatter(data[ax1name][cl==x],
            data[ax2name][cl==x],
            color=colour[x], marker='o', label='cluster ' + str(x+1))
    plt.legend()
    plt.xlabel(ax1name)
    plt.ylabel(ax2name)
    plt.title(str(k) + '-means Clustering On All Data')
    plt.plot()
    (r, p) = scipy.stats.pearsonr(data[ax1name], data[ax2name])
    print('r: {r}, p: {p}'.format(r=r, p=p))

In [ ]:
k = 5
df_deprivation_clusters = clusterise(df_deprivation2, k)

In [ ]:
plotClusters(df_deprivation_clusters, 'IMD - Average score', 'Income - Average score', k)

In [ ]:
plotClusters(df_deprivation_clusters, 'IMD - Average score', 'Employment - Average score', k)

In [ ]:
plotClusters(df_deprivation_clusters, 'IMD - Average score', 'Education, Skills and Training - Average score', k)

In [ ]:
plotClusters(df_deprivation_clusters, 'IMD - Average score', 'Health Deprivation and Disability - Average score', k)

In [ ]:
plotClusters(df_deprivation_clusters, 'IMD - Average score', 'Barriers to Housing and Services - Average score', k)

In [ ]:
plotClusters(df_deprivation_clusters, 'IMD - Average score', 'Living Environment - Average score', k)

In [ ]:
plotClusters(df_deprivation_clusters, 'IMD - Average score', 'Crime - Average score', k)

Investigating clustering on the deprivation data reveals a potential 5 types of deprived area:

- Overall low deprivation (low scores on all fronts)
- Overall high deprivation (high scores on all fronts)
- Overall medium deprivation, higher crime (medium scores on all fronts)
- High scores in living & barriers, low scores in education, employment & health
- Low scores in living & barriers, high scores in education, employment & health, lower crime

Lowering the number of clusters to 3, instead of reducing this to a simple low, high and meduim deprivation, emphasises the split in types 4 and 5. Raising the number of clusters reveals no extra significant information.  The clusters are not precisely repeatable, but the same five categories appears each time.

I have a hunch that this represents an urban vs rural split.  Where are the regions?

In [ ]:
import folium
import requests


Importing border data via http request is going to be slow.  It would be best to retrieve all the borders once and store them for future use.

In [ ]:
def addRegion(col, region):
    endpoint = ('https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/Local_Authority_Districts_December_2018_Boundaries_GB_BFC/FeatureServer/0/query?where=LAD18CD%3D\'' 
            + 
           region
            +'\'')
    params = {'f' : 'pgeojson' }

    try:
        r = requests.get(endpoint, params=params)
        r.raise_for_status()
        col[region] = r.json()
        if col[region].get('type') != 'FeatureCollection':
            print("No usable data at: " + endpoint)
            del col[region]
            return 0
        return 1
    except requests.exceptions.RequestException as e:
        print(e)
        return 0
    

In [ ]:
def addCounty(col, region):
    endpoint = ('https://opendata.arcgis.com/datasets/0b2c8acda82d4dd3aecdad8233d6bd9b_0.geojson?where=cty19cd%3D\'' 
            + 
           region
            +'\'')
    params = {'f' : 'pgeojson' }

    try:
        r = requests.get(endpoint, params=params)
        r.raise_for_status()
        col[region] = r.json()
        if col[region].get('type') != 'FeatureCollection':
            del col[region]
            print("No usable data at: " + endpoint)
            return 0
        return 1
    except requests.exceptions.RequestException as e:
        print(e)
        return 0
    


In [ ]:
def addAllRegions(col, regions):
    suc = 0
    for re in regions:  
        if re.find("E1") == -1:
            suc += addRegion(col, re)
        else:
            suc += addCounty(col, re)
    print(str(suc) + ' regions added of ' + str(len(regions)) + '.')
        
    

In [ ]:
##code commented to prevent 2nd run 

#col = {}
#addAllRegions(col, df_compare['Region #'])
#len(col)


For some reason I can't get Dorset for now, even trying alternative methods.  But I can save the rest.


**NOTE: mapdata file was too large to include in EMA submission.  It can be recreated with the notebook 
Data Acquisition And Cleaning**  

In [ ]:
#commented to prevent overwrite
#np.save('mapdata.npy', col) 

#Load 
#mapdata = np.load('mapdata.npy',allow_pickle='TRUE').item()

In [ ]:
def addMapRegions(regionDict, keys, style, m):
    for key in keys: 
        try:
            folium.GeoJson(regionDict[key], name=key, style_function=lambda x:style).add_to(m)
        except:
            print("An exception occurred with key: " + key)    

In [ ]:
m = folium.Map(location=[50, -3], width=500,
               height=800, zoom_start=5)

blue = {'fillColor': '#0000FF', 'color': '#0000FF'}
red = {'fillColor': '#FF0000', 'color': '#FF0000'}
green = {'fillColor': '#00FF00', 'color': '#00FF00'}
magenta = {'fillColor': '#FF00FF', 'color': '#FF00FF'}
cyan = {'fillColor': '#00FFFF', 'color': '#00FFFF'}

key = 'E10000009'
folium.GeoJson(mapdata[key], name=key, style_function=lambda x:cyan).add_to(m)
m

Something about adding all regions is causing a crash.  It's probably too much at once.  Try splitting it up.

In [ ]:
counties = df_compare['Region #'][df_compare['Region #'].str.contains("E1")]
len(counties)

In [ ]:
noncounties = set(df_compare['Region #']) - set(counties)
len(noncounties)

In [ ]:
m = folium.Map(location=[50, -3], width=500, height=800, zoom_start=5)

addMapRegions(mapdata, counties, blue, m)
m

We get an error looking for dorset, because it isn't there in the map data, but everything else displays. [Dorset data later found and added manually]

In [ ]:
#addMapRegions(mapdata, noncounties, red, m)
#m

OK, adding them both separately also causes a crash.  Just too much on one map.

In [ ]:
m2 = folium.Map(location=[50, -3], width=500, height=800, zoom_start=5)

addMapRegions(mapdata, noncounties, red, m2)
m2

Overlaying the two maps in an image editor shows that only Dorset is missing, everything else matches up without overlaps.

We should be able to merge to add the clusters:

In [ ]:
test = df_compare.merge(df_deprivation_clusters)
test

That worked, so do it on the main dataframe:

In [ ]:
df_compare = df_compare.merge(df_deprivation_clusters)
df_compare

NOTE: rerunning will change cluster numbering and exact members, but approximately the same clusters will emerge. For the clusters used for the rest of the notebook, run the folowing code to load the saved file:


In [ ]:
df_compare = pd.read_csv('clusters_diary.csv')

In [ ]:
cluster1 = df_compare["Region #"].loc[df_compare['Cluster']==0]
cluster2 = df_compare["Region #"].loc[df_compare['Cluster']==1]
cluster3 = df_compare["Region #"].loc[df_compare['Cluster']==2]
cluster4 = df_compare["Region #"].loc[df_compare['Cluster']==3]
cluster5 = df_compare["Region #"].loc[df_compare['Cluster']==4]


In [ ]:
print(len(cluster1))
print(len(cluster2))
print(len(cluster3))
print(len(cluster4))
print(len(cluster5))

In [ ]:
m3 = folium.Map(location=[50, -3], width=500, height=800, zoom_start=5)

addMapRegions(mapdata, cluster1, red, m3)
m3

In [ ]:
m3 = folium.Map(location=[50, -3], width=500, height=800, zoom_start=5)

addMapRegions(mapdata, cluster2, blue, m3)
m3

In [ ]:
m3 = folium.Map(location=[50, -3], width=500, height=800, zoom_start=5)

addMapRegions(mapdata, cluster3, green, m3)
m3

In [ ]:
m4 = folium.Map(location=[50, -3], width=500, height=800, zoom_start=5)

addMapRegions(mapdata, cluster4, cyan, m4)
m4

In [ ]:
m5 = folium.Map(location=[50, -3], width=500, height=800, zoom_start=5)

addMapRegions(mapdata, cluster5, magenta, m5)
m5

- Cluster 4, ~19 regions: overall high deprivation. Mostly midlands/north urban areas.
- Cluster 5, ~53 regions: overall low deprivation. Mostly rural south and midlands.
- Cluster 2, ~32 regions: medium deprivation at all levels.
- Cluster 1, ~20 regions: high scores in living & barriers, low scores in education, employment & health. Mostly London areas.
- Cluster 3, ~27 regions: Low scores in living & barriers, high scores in education, employment & health


How does overall deprivation relate to these clusters?

In [ ]:
plotClusters(df_compare, 'Cluster', 'IMD - Average score', k)

There is a clear group of high and low score ranges, with clusters 5 and 4 barely overlapping.  Clusters 2, 3 and 4 all represent the same range of scores that could be considered medium deprivation, though it can be noted that they overlap more with high deprivation than low deprivation.

Now that is done, we can compare KS4 data and overall deprivation.

## % 5+ A*-C

In [ ]:
plotClusters(df_compare, 'IMD - Average score', '% 5+ A*-C', k)

In [ ]:
plotClusters(df_compare, 'Education, Skills and Training - Average score', '% 5+ A*-C', k)

In [ ]:
plotClusters(df_compare, 'Cluster', '% 5+ A*-C', k)

% 5+ A*-C grades has a reasonable correlation with overall deprivation. When plotted against the clusters, it shows that high deprivation areas are clearly achieving less and low deprivation more, but the gap between medium and low deprivation areas is very small.  These clusters appear to stand up at least a little when applied to the second data set.  Plotting against Education etc deprivation produces an expected far higher corellation, and shows the overlap betwen clusters 1 (low overall deprivation) and cluster 5 (low deprivation in education, employment and health.

Outliers: 

- Isles of Scilly, cluster 1: >84%
- Knowsley, cluster 4: <46%
- Derby and Isle of Wight, cluster 2: <60%

### Outlier finding

In [ ]:
df_compare[df_compare['% 5+ A*-C'] > 84]

In [ ]:
df_compare[df_compare['% 5+ A*-C'] < 46]

In [ ]:
d = df_compare[df_compare['Cluster'] == 1]
d[d['% 5+ A*-C'] < 60]

## % 5+ A*-C inc Eng & Maths GCSE

In [ ]:
plotClusters(df_compare, 'IMD - Average score', '% 5+ A*-C inc Eng & Maths GCSE', k)

In [ ]:
plotClusters(df_compare, 'Education, Skills and Training - Average score', '% 5+ A*-C inc Eng & Maths GCSE', k)

In [ ]:
plotClusters(df_compare, 'Cluster', '% 5+ A*-C inc Eng & Maths GCSE', k)

Plotting the data for only those who achieved at the A*-C level including Maths and English shows a higher correlation, and a clearer gap between high and low deprivation.

Outliers: 

- Isles of Scilly, cluster 1: >73%
- Knowsley, cluster 4: <40%
- Southend-on-Sea: >64%
- Slough: >65%

### Outlier Finding

In [ ]:
d = df_compare[df_compare['Cluster'] == 0]
d[d['% 5+ A*-C inc Eng & Maths GCSE'] > 73]

In [ ]:
df_compare[df_compare['% 5+ A*-C inc Eng & Maths GCSE'] < 40]

In [ ]:
d = df_compare[df_compare['Cluster'] == 1]
d[d['% 5+ A*-C inc Eng & Maths GCSE'] > 64]

## % 5+ A*-C excl Eng & Maths GCSE

In [ ]:
plotClusters(df_compare, 'IMD - Average score', '% 5+ A*-C excl Eng & Maths GCSE', k)

In [ ]:
plotClusters(df_compare, 'Education, Skills and Training - Average score', '% 5+ A*-C excl Eng & Maths GCSE', k)

In [ ]:
plotClusters(df_compare, 'Cluster', '% 5+ A*-C excl Eng & Maths GCSE', k)

There is no pattern in the data for achievement of high grades excluding Maths and English.

## % 5+ A*-G

In [ ]:
plotClusters(df_compare, 'IMD - Average score', '% 5+ A*-G', k)

In [ ]:
plotClusters(df_compare, 'Education, Skills and Training - Average score', '% 5+ A*-G', k)

In [ ]:
plotClusters(df_compare, 'Cluster', '% 5+ A*-G', k)

The data for achievement in the 5+ A*-G is a slightly weaker correlation than the A-C range, and shows a reasonably similar range across all groups except the most deprived which is lower.  A pattern of two extreme outliers, one in cluster 1 and one in cluster 4, is emerging from multiple plots.  Both the cluster plot and the education deprivation plot again emphasis an overlap between cluster 1 and 5.

Outliers: 
- Isles of Scilly, cluster 1: 100%
- Knowsley, cluster 4: 85%

### Outlier finding

In [ ]:
df_compare[df_compare['% 5+ A*-G'] > 98]

In [ ]:
df_compare[df_compare['% 5+ A*-G'] < 86]

## % 5+ A*-G inc Eng & Maths GCSE

In [ ]:
plotClusters(df_compare, 'IMD - Average score', '% 5+ A*-G inc Eng & Maths GCSE', k)

In [ ]:
plotClusters(df_compare, 'Education, Skills and Training - Average score', '% 5+ A*-G inc Eng & Maths GCSE', k)

In [ ]:
plotClusters(df_compare, 'Cluster', '% 5+ A*-G inc Eng & Maths GCSE', k)

Again including only those who achieved 5+ passes including maths and English strengthens the correllation, and again achievement is mostly even across clusters except for the mot deprived. 

Outliers: 
- Isles of Scilly, cluster 1: 100%
- Knowsley, cluster 4: <82%
- Rochdale, cluster 3: <85% 
- Bexley, cluster 5: <87%

### Outlier finding

In [ ]:
df_compare[df_compare['% 5+ A*-G inc Eng & Maths GCSE'] > 98]

In [ ]:
df_compare[df_compare['% 5+ A*-G inc Eng & Maths GCSE'] < 85]

In [ ]:
d = df_compare[df_compare['Cluster'] == 4]
d[d['% 5+ A*-G inc Eng & Maths GCSE'] < 87]

## % 5+ A*-G excl Eng & Maths GCSE

In [ ]:
plotClusters(df_compare, 'IMD - Average score', '% 5+ A*-G excl Eng & Maths GCSE', k)

In [ ]:
plotClusters(df_compare, 'Education, Skills and Training - Average score', '% 5+ A*-G excl Eng & Maths GCSE', k)

In [ ]:
plotClusters(df_compare, 'Cluster', '% 5+ A*-G excl Eng & Maths GCSE', k)

The percentage achieving 5+ passes *without* English and Maths shows a small positive correlation with overall deprivation.  The cluster graph shows things to be relatively similar across groups, though the most and least deprived have a small but clear difference position. 

Outliers: 
- Isles of Scilly, cluster 1: 0%
- Rochdale, cluster 3: >6% 
- Bexley, cluster 5: >6%
- Salford, cluster 4: >6%
- Coventry & Enfield, cluster 2: >4%


### Outlier Finding

In [ ]:
df_compare[df_compare['% 5+ A*-G excl Eng & Maths GCSE'] == 0]

In [ ]:
df_compare[df_compare['% 5+ A*-G excl Eng & Maths GCSE'] > 6]

In [ ]:
d = df_compare[df_compare['Cluster'] == 1]
d[d['% 5+ A*-G excl Eng & Maths GCSE'] > 4]

### GCSE or Equv. Maths & English % entered

In [ ]:
plotClusters(df_compare, 'IMD - Average score', 'GCSE or Equv. Maths & English % entered', k)

In [ ]:
plotClusters(df_compare, 'Education, Skills and Training - Average score', 'GCSE or Equv. Maths & English % entered', k)

In [ ]:
plotClusters(df_compare, 'Cluster', 'GCSE or Equv. Maths & English % entered', k)

A similar pattern emerges again: weak negative correlation, low scores for the most deprived, but the same y axis range for least and medium deprivation areas. 

Outliers: 
- Isles of Scilly, cluster 1: 100%
- Rochdale, cluster 3: < 90% 
- Bexley, cluster 5: < 92%
- Salford, cluster 4: < 92%
- Knowsley, cluster 4: < 92%

### Outlier Finding

In [ ]:
df_compare[df_compare['GCSE or Equv. Maths & English % entered'] == 100]

In [ ]:
df_compare[df_compare['GCSE or Equv. Maths & English % entered'] < 92]

### GCSE or Equv. Maths & English A*-C % achieved

In [ ]:
plotClusters(df_compare, 'IMD - Average score', 'GCSE or Equv. Maths & English A*-C % achieved', k)

In [ ]:
plotClusters(df_compare, 'Education, Skills and Training - Average score', 'GCSE or Equv. Maths & English A*-C % achieved', k)

In [ ]:
plotClusters(df_compare, 'Cluster', 'GCSE or Equv. Maths & English A*-C % achieved', k)

Achievement of A*-C in English is most negatively corellated with IMD score, though not by significantly more than 5+ GSCEs in all pass ranges.  Again the difference between least and most deprived is clear.

Outliers: 
- Isles of Scilly, cluster 1: >70%
- Knowsley, cluster 4: < 40%
- Southend-on-Sea, cluster 2: >64%
- Slough, cluster 2: >65%
- Kingston upon Thames, cluster 5: >75% 

### Outlier Finding

In [ ]:
d = df_compare[df_compare['Cluster'] == 0]
d[d['GCSE or Equv. Maths & English A*-C % achieved'] > 70]

In [ ]:
df_compare[df_compare['GCSE or Equv. Maths & English A*-C % achieved'] < 40]

In [ ]:
d = df_compare[df_compare['Cluster'] == 1]
d[d['GCSE or Equv. Maths & English A*-C % achieved'] > 65]

In [ ]:
d = df_compare[df_compare['Cluster'] == 4]
d[d['GCSE or Equv. Maths & English A*-C % achieved'] > 75]

### Outlier Investigation

The isles of Isles of Scilly stand out for high attainment overall, and Knowsley for low attainment overall.  Southend-on-Sea and Slough do notably better than expected for KS4 acheivement in the region of high attainment given their group.  Rochdale, Bexley, and Salford underperform for their group, particularly in the area of KS4 entry.

In [ ]:
outliers = ['E06000053', 'E08000011', 'E06000033', 'E06000039', 'E08000005', 'E08000006', 'E09000004']

In [ ]:
outliers

In [ ]:
m = folium.Map(location=[50, -3], width=500, height=800, zoom_start=5)

addMapRegions(mapdata, outliers, blue, m)
m

Clear geographical clustering. 

In [ ]:
noOutliers = df_compare.drop(df_compare[df_compare["Region #"].isin(outliers)].index)
noOutliers

In [ ]:
plotClusters(noOutliers, 'IMD - Average score', '% 5+ A*-C', k)

Weaker corellation

In [ ]:
plotClusters(noOutliers, 'IMD - Average score', '% 5+ A*-C inc Eng & Maths GCSE', k)

In [ ]:
plotClusters(noOutliers, 'IMD - Average score', '% 5+ A*-C excl Eng & Maths GCSE', k)

In [ ]:
plotClusters(noOutliers, 'IMD - Average score', '% 5+ A*-G', k)

In [ ]:
plotClusters(noOutliers, 'IMD - Average score', '% 5+ A*-G inc Eng & Maths GCSE', k)

In [ ]:
plotClusters(noOutliers, 'IMD - Average score', '% 5+ A*-G excl Eng & Maths GCSE', k)

In [ ]:
plotClusters(noOutliers, 'IMD - Average score', 'GCSE or Equv. Maths & English % entered', k)

In [ ]:
plotClusters(noOutliers, 'IMD - Average score', 'GCSE or Equv. Maths & English A*-C % achieved', k)